In [0]:
!pip install http://download.pytorch.org/whl/cu92/torch-0.4.1-cp36-cp36m-linux_x86_64.whl

In [0]:
!pip install torchvision

  Using cached https://files.pythonhosted.org/packages/85/5e/e91792f198bbc5a0d7d3055ad552bc4062942d27eaf75c3e2783cf64eae5/Pillow-5.4.1-cp36-cp36m-manylinux1_x86_64.whl
  Found existing installation: Pillow 4.0.0
    Uninstalling Pillow-4.0.0:
      Successfully uninstalled Pillow-4.0.0


In [0]:
!git clone https://github.com/magus96/BCDCNN_2.git

fatal: destination path 'BCDCNN_2' already exists and is not an empty directory.


In [0]:
cd '/content/BCDCNN_2/'

/content/BCDCNN_2


In [0]:
!pip install Pillow==4.0.0
!pip install PIL
!pip install image

  Using cached https://files.pythonhosted.org/packages/37/e8/b3fbf87b0188d22246678f8cd61e23e31caa1769ebc06f1664e2e5fe8a17/Pillow-4.0.0-cp36-cp36m-manylinux1_x86_64.whl
torchvision 0.2.1 has requirement pillow>=4.1.1, but you'll have pillow 4.0.0 which is incompatible.
  Found existing installation: Pillow 5.4.1
    Uninstalling Pillow-5.4.1:
      Successfully uninstalled Pillow-5.4.1
  Could not find a version that satisfies the requirement PIL (from versions: )
No matching distribution found for PIL


In [0]:

import cv2
import os.path
import pickle
import matplotlib.pyplot as plt
from imutils import paths
from sklearn.preprocessing import LabelBinarizer
import numpy as np
import torch
import torchvision
import torch.nn as nn
import torchvision.transforms as transforms
import torch.nn.functional as f
from torch.autograd import Variable
from sklearn import metrics
from sklearn.metrics import roc_curve,auc
from sklearn.model_selection import train_test_split
import torch.optim as optim
from torch.autograd import Variable

In [0]:
transform=transforms.Compose([
    transforms.Resize((128,128)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])
train_dataset=torchvision.datasets.ImageFolder(
    root="/content/BCDCNN_2/Data/",
    transform=transform
)
train_loader=torch.utils.data.DataLoader(
    train_dataset,batch_size=32,num_workers=1,shuffle=True)

test_dataset=torchvision.datasets.ImageFolder(
    root="/content/BCDCNN_2/Validation/",
    transform=transform
)
test_loader=torch.utils.data.DataLoader(
    test_dataset,batch_size=32,num_workers=1,shuffle=True)

       
    
    

In [0]:
class Bnet(nn.Module):
  def __init__(self):
    super(Bnet,self).__init__()
    self.layer1=nn.Sequential(
        nn.Conv2d(3,64,kernel_size=3,stride=1,padding=1),
        nn.ReLU(),
        nn.Conv2d(64,64,kernel_size=3,stride=1,padding=1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2,stride=2))
    self.layer2=nn.Sequential(
        nn.Conv2d(64,128,kernel_size=3,stride=1,padding=1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2,stride=2))
    self.layer3=nn.Sequential(
        nn.Conv2d(128,256,kernel_size=3,stride=1,padding=1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2,stride=2),
        nn.MaxPool2d(kernel_size=2,stride=2))
    self.layer4=nn.Sequential(
        nn.Conv2d(256,512,kernel_size=3,stride=1,padding=1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2,stride=2))
    self.dropout=nn.Dropout(0.2)
    self.fc1=nn.Linear(512*4*4,256)
    self.fc2=nn.Linear(256,128)
    self.fc3=nn.Linear(128,2)
    self.softmax=nn.LogSoftmax(dim=1)
  def forward(self,x):
    out=self.layer1(x)
    out=self.layer2(out)
    out=self.layer3(out)
    out=self.layer4(out)
    out=out.view(-1,512*4*4)
    out=f.relu(self.fc1(out))
    out=self.dropout(f.relu(self.fc2(out)))
    out=self.softmax(self.fc3(out))
    return out
  
model=Bnet()
device=torch.device("cuda:0")
model.to(device)
model_params=list(model.parameters())
criterion=nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(model.parameters(),lr=0.001)
print(model.parameters)

<bound method Module.parameters of Bnet(
  (layer1): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer3): Sequential(
    (0): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer4): Sequential(
    (0): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, d

In [0]:
model.train()
for epoch in range(20):
  running_loss=0.0
  for i,data in enumerate(train_loader):
    inputs,target=data
    inputs,target=inputs.to(device),target.to(device)
    optimizer.zero_grad()
    outputs=model(inputs)
    loss=criterion(outputs,target)
    loss.backward()
    optimizer.step()
    running_loss+=loss.item()
print(running_loss)
print("finished training")


2.5209309263154864
finished training


In [0]:
def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

In [0]:
correct = 0
total = 0
with torch.no_grad():
    for data in test_loader:
        images, labels = data
        images,labels=images.to(device),labels.to(device)
        outputs = model(images)
        loss=criterion(outputs,labels)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print("testing loss=")
print(loss.item())
print('Accuracy of the network on 398 test images: %d %%' % (
    100 * correct / total))        

testing loss=
0.36315134167671204
Accuracy of the network on 398 test images: 96 %


In [0]:
path='/content/BCDCNN_2/gg1'
torch.save(model.state_dict(),path)